In [1]:
import numpy as np
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt

# Load the fine-tuned Xception model
model = tf.keras.models.load_model('xception_model.h5')

# Function to predict whether an input image is real or fake and return the real %
def predict_real_fake(image_path):
    # Preprocess the image
    img_array, original_image = preprocess_image(image_path)
    predictions = model.predict(img_array)
    real_prob = predictions[0][0]
    return real_prob * 100  # Return percentage for real

# Function to generate a heatmap for an input image using Grad-CAM
def generate_heatmap(image_path, layer_name='block14_sepconv2'):
    img_array, original_image = preprocess_image(image_path)
    heatmap = grad_cam(model, img_array, layer_name)
    overlayed_image = overlay_heatmap(heatmap, np.uint8(original_image))
    return overlayed_image, heatmap

# Function to plot the graph of real % for each frame in a video
def plot_frame_real_predictions(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_real_probs = []
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert frame to RGB and preprocess
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img_array = preprocess_image_from_array(frame_rgb)

        # Predict real probability for the frame
        predictions = model.predict(img_array)
        real_prob = predictions[0][0]

        frame_real_probs.append(real_prob * 100)
        frame_count += 1

    cap.release()

    # Plotting the graph
    plt.figure(figsize=(12, 6))
    plt.plot(range(1, frame_count + 1), frame_real_probs, label='Real % Prediction')
    plt.xlabel('Frame Number')
    plt.ylabel('Real %')
    plt.title('Real % Prediction Over Video Frames')
    plt.grid(True)
    plt.show()

# Helper function to preprocess the image
def preprocess_image(image_path, target_size=(299, 299)):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=target_size)
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = tf.keras.applications.xception.preprocess_input(img_array)
    return img_array, img

# Helper function to preprocess the image from an array (used for video frames)
def preprocess_image_from_array(image_array, target_size=(299, 299)):
    img_array = cv2.resize(image_array, target_size)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = tf.keras.applications.xception.preprocess_input(img_array)
    return img_array

# Grad-CAM implementation
def grad_cam(model, img_array, layer_name, pred_index=None):
    grad_model = tf.keras.models.Model(
        [model.inputs],
        [model.get_layer(layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        loss = predictions[:, pred_index]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # Normalize heatmap
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

# Overlay heatmap on the image
def overlay_heatmap(heatmap, image, alpha=0.4, colormap=cv2.COLORMAP_JET):
    heatmap = cv2.resize(heatmap, (image.shape[1], image.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, colormap)
    overlayed_img = cv2.addWeighted(heatmap, alpha, image, 1 - alpha, 0)
    return overlayed_img


In [2]:
# Predict real % for img1.png
real_prob_img1 = predict_real_fake('img4.png')
print(f"Predicted real % for img1.png: {real_prob_img1:.2f}%")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 921ms/step
Predicted real % for img1.png: 0.15%


In [18]:
import cv2
import numpy as np
import tensorflow as tf

# Load the pre-trained deepfake detection model (replace this with your model if available)
# This example uses Xception with ImageNet weights for demonstration purposes.
# Replace with your actual deepfake detection model.
model = tf.keras.models.load_model('xception_model.h5')
# model = tf.keras.applications.Xception(weights='imagenet')

# Load the face detection model from OpenCV
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Function to preprocess image for prediction
def preprocess_image(image, target_size=(299, 299)):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, target_size)
    image = image / 255.0  # Normalize pixel values
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    return image

# Function to draw a box with the prediction
def draw_prediction(image, x, y, w, h, prediction):
    if prediction < 0.5:
        # Red box for low confidence (below 50%)
        color = (0, 0, 255)
        label = f"Real: {100 * (1 - prediction):.2f}%"
    else:
        # Green box for high confidence (above 50%)
        color = (0, 255, 0)
        label = f"Fake: {100 * prediction:.2f}%"
    
    # Draw the rectangle around the face
    cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
    
    # Put the label on the box
    cv2.putText(image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

# Function to detect and predict deepfake status for faces in the image
def detect_and_predict(image_path):
    # Load the image
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces in the image
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        # Crop and preprocess the face for prediction
        face_img = image[y:y+h, x:x+w]
        face_array = preprocess_image(face_img)

        # Predict using the pre-trained model
        prediction = model.predict(face_array)[0][0]  # Adjust indexing based on the model output format

        # Draw the prediction on the image
        draw_prediction(image, x, y, w, h, prediction)

    # Display the image with the boxes and predictions
    cv2.imshow('Face Detection and Deepfake Prediction', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Example usage
if __name__ == "__main__":
    image_path = 'img1.png'  # Replace with your image path
    detect_and_predict(image_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
